# 🤖 Agentic Podcast Generator - Step by Step Execution

This notebook breaks down the execution of the Agentic Podcast Generator program into individual, executable steps. Each cell demonstrates a specific part of the workflow, from initialization to final output generation.

**✅ Compatible with Google Colab** - Automatically detects environment and configures appropriately.

## Overview

The Agentic Podcast Generator is a multi-agent system that:
1. Researches topics using Perplexity AI
2. Generates keywords and hashtags
3. Creates LinkedIn posts
4. Produces voice dialog scripts
5. Stores results in a SQLite database

All steps are executed sequentially in this notebook for educational purposes.

## 1. Load the Program Code

First, we need to load the necessary modules and set up the environment. This includes importing all the required libraries and loading environment variables.

In [5]:
# Load environment variables and imports
import os
import sys
import asyncio
import logging
from typing import Dict, Any, Optional

# Check if running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("🔧 Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("🔧 Running in local environment")

if IN_COLAB:
    # Use Colab secrets
    from google.colab import userdata
    
    # Set environment variables from Colab secrets
    os.environ['OPENROUTER_API_KEY'] = userdata.get('OPENROUTER_API_KEY')
    print("✅ API keys loaded from Colab secrets")
else:
    # Use local .env file
    from dotenv import load_dotenv
    load_dotenv()
    print("✅ Environment variables loaded from .env file")

# Add the project root to Python path
if IN_COLAB:
    # In Colab, the path will be set after cloning
    sys.path.insert(0, '/content/agentic-podcast-generator')
    print("✅ Colab path configured")
else:
    sys.path.insert(0, '/Users/rj/Programs/agentic-podcast-generator')
    print("✅ Local path configured")

🔧 Running in local environment
✅ Environment variables loaded from .env file
✅ Local path configured


### Colab Setup Instructions

If running in Google Colab, follow these steps:

1. **Set up secrets**: Go to Colab menu → Runtime → Secrets
   - Add a secret named `OPENROUTER_API_KEY` with your OpenRouter API key
   - Make sure "Notebook access" is enabled

2. **Clone the repository** (run this cell if in Colab):

In [6]:
# Clone repository in Colab (run this cell if in Colab)
if IN_COLAB:
    !git clone https://github.com/wubbyweb/agentic-podcast-generator.git
    %cd agentic-podcast-generator
    !pip install -r requirements.txt
    print("✅ Repository cloned and dependencies installed")
else:
    print("ℹ️  Local environment detected - skipping Colab setup")

ℹ️  Local environment detected - skipping Colab setup


## 2. Parse/Setup - Import Core Modules

Now we'll import the core modules of the agentic system including configuration, database, services, and agents.

In [2]:
# Import core modules
from config.settings import config
from database.connection import init_database
from services.logger import setup_logging
from services.openrouter_client import OpenRouterClient
from agents.master_agent import MasterAgent
from agents.sub_agents.keyword_generator import KeywordGenerator
from agents.sub_agents.post_generator import PostGenerator
from agents.sub_agents.voice_dialog import VoiceDialogGenerator

print("✅ Core modules imported successfully")

✅ Core modules imported successfully


## 3. Identify Key Execution Steps

The agentic system follows these main execution steps:

1. **Configuration Validation** - Ensure all required settings are present
2. **Logging Setup** - Initialize logging system
3. **Database Initialization** - Set up SQLite database
4. **Research Phase** - Get comprehensive research from Perplexity AI
5. **Agent Initialization** - Create and initialize all sub-agents
6. **Sequential Execution** - Run each sub-agent (normally parallel, but sequential here for demonstration)
7. **Results Compilation** - Gather and format all outputs
8. **Cleanup** - Close database connections and clean up resources

Let's execute each step individually.

### Step 3.1: Configuration Validation

Validate that all required configuration settings are present and valid.

In [4]:
# Step 3.1: Validate configuration
try:
    config.validate_config()
    print("✅ Configuration validation passed")
    print(f"   OpenRouter API Key: {'✓ Set' if config.openrouter_api_key else '✗ Missing'}")
    print(f"   Database URL: {config.database_url}")
    print(f"   Log Level: {config.log_level}")
except Exception as e:
    print(f"❌ Configuration validation failed: {e}")
    raise

✅ Configuration validation passed
   OpenRouter API Key: ✓ Set
   Database URL: sqlite+aiosqlite:///./agentic_system.db
   Log Level: INFO


### Step 3.2: Logging Setup

Initialize the logging system for the application.

In [ ]:
# Step 3.2: Setup logging
log_level = "INFO"  # Can be changed to DEBUG for more verbose output
setup_logging(level=log_level)

logger = logging.getLogger(__name__)
logger.info("Logging system initialized")

print("✅ Logging system initialized")

### Step 3.3: Database Initialization

Initialize the SQLite database and create necessary tables.

**Note for Colab users**: The database will be created in your Colab session storage. Files will persist during the session but will be lost when the session ends. For persistent storage, consider using Google Drive mounting.

In [ ]:
# Step 3.3: Initialize database
await init_database()
logger.info("Database initialized")

print("✅ Database initialized successfully")

### Step 3.4: Research Phase

Get comprehensive research from Perplexity AI for our topic. This is the foundation for all subsequent content generation.

In [ ]:
# Step 3.4: Research Phase
topic = "Artificial Intelligence in Healthcare"  # You can change this topic

async def get_perplexity_research(topic: str) -> str:
    """Get comprehensive research from Perplexity API using sonar model."""
    async with OpenRouterClient(config.openrouter_api_key) as client:
        messages = [
            {
                "role": "system",
                "content": "You are an expert research assistant. Provide comprehensive, up-to-date research and analysis on the given topic. Include current facts, key insights, trends, and relevant data points."
            },
            {
                "role": "user",
                "content": f"Research and analyze this topic comprehensively: {topic}. Provide detailed findings, current developments, and key insights."
            }
        ]

        response = await client.chat_completion(
            model="perplexity/sonar",
            messages=messages,
            max_tokens=4000,
            temperature=0.3
        )

        return client.extract_response_content(response)

print(f"🔍 Researching topic: {topic}")
research_response = await get_perplexity_research(topic)
logger.info("Research completed from Perplexity")

print("✅ Research completed successfully")
print(f"📄 Research length: {len(research_response)} characters")
print(f"📄 Research preview: {research_response[:200]}...")

### Step 3.5: Agent Initialization

Create and initialize all sub-agents that will process the research and generate content.

In [ ]:
# Step 3.5: Agent Initialization
from database.connection import create_session_record

# Create a session record
session_id = await create_session_record(topic)
print(f"✅ Created session {session_id} for topic: {topic}")

# Initialize sub-agents
keyword_agent = KeywordGenerator()
keyword_agent.session_id = session_id
await keyword_agent.__aenter__()

post_agent = PostGenerator()
post_agent.session_id = session_id
await post_agent.__aenter__()

voice_agent = VoiceDialogGenerator()
voice_agent.session_id = session_id
await voice_agent.__aenter__()

print("✅ All sub-agents initialized successfully")

## 4. Sequential Execution of Sub-Agents

Normally, all sub-agents run in parallel for efficiency. Here we'll execute them sequentially to demonstrate each step clearly.

### Step 4.1: Keyword Generator Execution

Generate SEO-optimized keywords and hashtags using the research data.

In [ ]:
# Step 4.1: Execute Keyword Generator
keyword_input = {
    "topic": topic,
    "research_response": research_response
}

print("🏷️ Generating keywords and hashtags...")
keyword_result = await keyword_agent.execute_with_logging(keyword_input)

print("✅ Keyword generation completed")
print(f"🔑 Keywords: {keyword_result.get('keywords', [])[:10]}")  # Show first 10
print(f"📊 Hashtags: {keyword_result.get('hashtags', [])[:10]}")  # Show first 10

### Step 4.2: Post Generator Execution

Create an engaging LinkedIn post based on the research and keywords.

In [ ]:
# Step 4.2: Execute Post Generator
research_results = {
    "topic": topic,
    "research_plan": {"search_queries": [topic], "source_types": ["perplexity"]},
    "results": [{
        "title": f"Perplexity Research: {topic}",
        "url": f"https://perplexity.ai/search?q={topic.replace(' ', '+')}",
        "snippet": research_response[:500],
        "content": research_response,
        "source": "perplexity",
        "relevance_score": 1.0,
        "credibility_score": 0.9
    }],
    "summary": research_response,
    "total_sources": 1,
    "credibility_score": 0.9
}

post_input = {
    "topic": topic,
    "research": research_results,
    "research_response": research_response
}

print("💼 Generating LinkedIn post...")
post_result = await post_agent.execute_with_logging(post_input)

print("✅ Post generation completed")
print("📝 Generated Post:")
print("-" * 50)
print(post_result.get("content", "No content generated"))
print("-" * 50)

### Step 4.3: Voice Dialog Generator Execution

Convert the LinkedIn post into a conversational voice dialog script.

In [ ]:
# Step 4.3: Execute Voice Dialog Generator
voice_input = {
    "topic": topic,
    "research_response": research_response
}

print("🎙️ Generating voice dialog script...")
voice_result = await voice_agent.execute_with_logging(voice_input)

print("✅ Voice dialog generation completed")
print("🎭 Generated Voice Script:")
print("-" * 50)
print(voice_result.get("dialog", "No dialog generated"))
print("-" * 50)

## 5. Results Compilation and Validation

Compile all the generated content into a final results dictionary and validate the outputs.

In [ ]:
# Step 5: Compile final results
final_results = {
    "session_id": session_id,
    "topic": topic,
    "research_response": research_response,
    "linkedin_post": post_result.get("content", ""),
    "voice_dialog": voice_result.get("dialog", ""),
    "keywords": keyword_result.get("keywords", []),
    "hashtags": keyword_result.get("hashtags", []),
    "research_summary": research_response[:500]
}

print("✅ Results compiled successfully")
print("\n📊 Final Results Summary:")
print(f"   Session ID: {final_results['session_id']}")
print(f"   Topic: {final_results['topic']}")
print(f"   Keywords Generated: {len(final_results['keywords'])}")
print(f"   Hashtags Generated: {len(final_results['hashtags'])}")
print(f"   LinkedIn Post Length: {len(final_results['linkedin_post'])} characters")
print(f"   Voice Dialog Length: {len(final_results['voice_dialog'])} characters")
print(f"   Research Summary Length: {len(final_results['research_summary'])} characters")

## 6. Cleanup and Best Practices

Clean up resources and update the session status. This demonstrates proper resource management.

In [ ]:
# Step 6: Cleanup and finalize
from database.connection import update_session_status

# Update session status to completed
await update_session_status(session_id, "completed")
print(f"✅ Session {session_id} marked as completed")

# Clean up agents
await keyword_agent.__aexit__(None, None, None)
await post_agent.__aexit__(None, None, None)
await voice_agent.__aexit__(None, None, None)
print("✅ All agents cleaned up")

print("\n🎉 Agentic Podcast Generator execution completed successfully!")
print("\n💡 Best Practices Demonstrated:")
print("   • Proper async/await usage")
print("   • Resource management with context managers")
print("   • Database session management")
print("   • Error handling and logging")
print("   • Sequential execution for debugging")
print("   • Result validation and compilation")

## Conclusion

This notebook has successfully demonstrated the complete execution flow of the Agentic Podcast Generator system. By breaking down the program into individual, executable steps, we've shown:

1. **System Initialization**: Loading modules, validating configuration, setting up logging and database
2. **Research Phase**: Using Perplexity AI to gather comprehensive topic research
3. **Agent Orchestration**: Sequential execution of sub-agents for content generation
4. **Result Compilation**: Gathering and validating all generated outputs
5. **Resource Management**: Proper cleanup and session management

The system generates:
- **Keywords & Hashtags**: SEO-optimized content tags
- **LinkedIn Posts**: Professional social media content
- **Voice Dialog Scripts**: Conversational podcast scripts
- **Research Summaries**: Condensed topic insights

All outputs are stored in a SQLite database for persistence and can be retrieved for future use or analysis.

### Colab Usage Notes

- **Secrets Setup**: Configure your `OPENROUTER_API_KEY` in Colab secrets before running
- **Repository**: The notebook automatically clones the repository when run in Colab
- **Persistence**: Database files are stored in session storage and will be lost when the session ends
- **Dependencies**: All required packages are automatically installed

### Running with Different Topics

To run this notebook with a different topic, simply change the `topic` variable in Step 3.4 and re-run all cells from that point forward.

### Local vs Colab Execution

The notebook automatically detects the environment and configures itself appropriately:
- **Local**: Uses `.env` files and local paths
- **Colab**: Uses secrets and clones the repository